# Lab | LangChain Med

## Objectives

- continue on with lesson 2' example, use different datasets to test what we did in class. Some datasets are suggested in the notebook but feel free to scout other datasets on HuggingFace or Kaggle.
- Find another model on Hugging Face and compare it.
- Modify the prompt to fit your selected dataset.

In [1]:
import numpy as np 
import pandas as pd

## Load the Dataset
As you can see the notebook is ready to work with three different Datasets. Just uncomment the lines of the Dataset you want to use. 

I selected Datasets with News. Two of them have just a brief decription of the news, but the other contains the full text. 

As we are working in a free and limited space, I limited the number of news to use with the variable MAX_NEWS. Feel free to pull more if you have memory available. 

The name of the field containing the text of the new is stored in the variable *DOCUMENT* and the metadata in *TOPIC*

In [6]:
# news = pd.read_csv('/kaggle/input/topic-labeled-news-dataset/labelled_newscatcher_dataset.csv', sep=';')
# MAX_NEWS = 1000
# DOCUMENT="title"
# TOPIC="topic"

#news = pd.read_csv('/kaggle/input/bbc-news/bbc_news.csv')
#MAX_NEWS = 1000
#DOCUMENT="description"
#TOPIC="title"

#news = pd.read_csv('/kaggle/input/mit-ai-news-published-till-2023/articles.csv')
#MAX_NEWS = 100
#DOCUMENT="Article Body"
#TOPIC="Article Header"

news = pd.read_csv(r'C:\Users\Zahinos\Desktop\AI\Week7\Day5\lab-langchain-med\datasets\bbc_news.csv') #Ideally pick one from the commented ones above
MAX_NEWS = 1000
DOCUMENT="description"
TOPIC="title"

ChromaDB requires that the data has a unique identifier. We can make it with this statement, which will create a new column called **Id**.


In [7]:
news["id"] = news.index
news.head()

,title,pubDate,guid,link,description,id
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...,0
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as...",1
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...,2
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...,3
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...,4


In [8]:
#Because it is just a course we select a small portion of News.
subset_news = news.head(MAX_NEWS)

## Import and configure the Vector Database
I'm going to use ChromaDB, the most popular OpenSource embedding Database. 

First we need to import ChromaDB, and after that import the **Settings** class from **chromadb.config** module. This class allows us to change the setting for the ChromaDB system, and customize its behavior. 

In [9]:
!pip install chromadb

In [10]:
import chromadb
from chromadb.config import Settings

Now we need to create the seetings object calling the **Settings** function imported previously. We store the object in the variable **settings_chroma**.

Is necessary to inform two parameters 
* chroma_db_impl. Here we specify the database implementation and the format how store the data. I choose ***duckdb***, because his high-performace. It operate primarly in memory. And is fully compatible with SQL. The store format ***parquet*** is good for tabular data. With good compression rates and performance. 

* persist_directory: It just contains the directory where the data will be stored. Is possible work without a directory and the data will be stored in memory without persistece, but Kaggle dosn't support that. 

In [11]:
chroma_client = chromadb.PersistentClient(path="/path/to/persist/directory")

## Filling and Querying the ChromaDB Database
The Data in ChromaDB is stored in collections. If the collection exist we need to delete it. 

In the next lines, we are creating the collection by calling the ***create_collection*** function in the ***chroma_client*** created above.

In [12]:
collection_name = "news_collection"
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
        chroma_client.delete_collection(name=collection_name)

collection = chroma_client.create_collection(name=collection_name)
    

It's time to add the data to the collection. Using the function ***add*** we need to inform, at least ***documents***, ***metadatas*** and ***ids***. 
* In the **document** we store the big text, it's a different column in each Dataset. 
* In **metadatas**, we can informa a list of topics. 
* In **id** we need to inform an unique identificator for each row. It MUST be unique! I'm creating the ID using the range of MAX_NEWS. 


In [13]:

collection.add(
    documents=subset_news[DOCUMENT].tolist(),
    metadatas=[{TOPIC: topic} for topic in subset_news[TOPIC].tolist()],
    ids=[f"id{x}" for x in range(MAX_NEWS)],
)

C:\Users\Zahinos\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:03<00:00, 24.1MiB/s]


In [14]:
results = collection.query(query_texts=["laptop"], n_results=10 )

print(results)

{'ids': [['id775', 'id707', 'id310', 'id587', 'id444', 'id751', 'id701', 'id862', 'id191', 'id740']], 'embeddings': None, 'documents': [['Photography student Thorsten Mjölnir captures the way students decorate their laptops.', 'Why sales of very basic mobile phones, without apps and internet connection, are increasing.', "What do you do when your collection of millions of books keeps growing but your bookshelves don't?", 'The developers of a powerful mini aircraft hope it will be used by the armed forces.', 'How tech is helping young families and couples regain their busy social lives after Covid.', 'Watch as Lee Zii Jia of Malaysia records a speed of 372km/h on his backhand point against Lakshya Sen of India in the All England Badminton Championships.', 'The Royal Mint has found a way to turn old circuit boards from phones, computers and TVs into gold.', 'A van was reportedly hijacked and driven to the venue, and a controlled explosion has since been carried out.', 'The Ukrainian pres

## Vector MAP

In [15]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [16]:

getado = collection.get(ids="id141", 
                       include=["documents", "embeddings"])


In [17]:
word_vectors = getado["embeddings"]
word_list = getado["documents"]
word_vectors

array([[ 8.36053193e-02,  9.88176279e-03, -6.33521751e-02,
         6.83554960e-03,  9.83688012e-02,  6.62363842e-02,
         8.81884769e-02,  4.32013953e-03,  4.62584458e-02,
        -2.53718700e-02,  3.03161126e-02,  2.90436596e-02,
         3.73522788e-02,  1.74557697e-02, -3.08765993e-02,
        -4.47610579e-02, -7.60653382e-03, -5.80879115e-02,
        -4.53988910e-02, -6.97019175e-02, -3.50682274e-03,
         4.58132587e-02,  1.58201139e-02,  3.44685316e-02,
        -6.20314032e-02,  9.38326642e-02,  5.95623329e-02,
        -3.07523180e-02, -8.41464475e-02, -2.55898647e-02,
        -5.44909714e-03, -1.64661184e-02, -1.00670859e-01,
         5.22239953e-02, -6.52278960e-03,  6.44802079e-02,
         6.21989220e-02, -4.69207810e-03, -3.38835940e-02,
        -5.86173963e-04, -9.11896750e-02, -9.24299806e-02,
        -4.39610668e-02, -4.71336246e-02,  1.49744772e-03,
        -9.09678731e-03,  2.79689636e-02, -1.05015799e-01,
         6.03262410e-02,  5.91969416e-02, -2.59885769e-0

Once we have our information inside the Database we can query It, and ask for data that matches our needs. The search is done inside the content of the document, and it dosn't look for the exact word, or phrase. The results will be based on the similarity between the search terms and the content of documents. 

The metadata is not used in the search, but they can be utilized for filtering or refining the results after the initial search. 


## Loading the model and creating the prompt
TRANSFORMERS!!
Time to use the library **transformers**, the most famous library from [hugging face](https://huggingface.co/) for working with language models. 

We are importing: 
* **Autotokenizer**: It is a utility class for tokenizing text inputs that are compatible with various pre-trained language models.
* **AutoModelForCasualLLM**: it provides an interface to pre-trained language models specifically designed for language generation tasks using causal language modeling (e.g., GPT models), or the model used in this notebook ***databricks/dolly-v2-3b***.
* **pipeline**: provides a simple interface for performing various natural language processing (NLP) tasks, such as text generation (our case) or text classification. 

The model selected is [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b), the smallest Dolly model. It have 3billion paramaters, more than enough for our sample, and works much better than GPT2. 

Please, feel free to test [different Models](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending), you need to search for NLP models trained for text-generation. My recomendation is choose "small" models, or we will run out of memory in kaggle.  


In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "databricks/dolly-v2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)



tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

c:\Users\Zahinos\Desktop\AI\Week5\Day5(4)\lab-qlora-tuning-peft\.conda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Zahinos\.cache\huggingface\hub\models--databricks--dolly-v2-3b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

The next step is to initialize the pipeline using the objects created above. 

The model's response is limited to 256 tokens, for this project I'm not interested in a longer response, but it can easily be extended to whatever length you want.

Setting ***device_map*** to ***auto*** we are instructing the model to automaticaly select the most appropiate device: CPU or GPU for processing the text generation.  

In [19]:
pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto",
)

Device set to use cpu


## Creating the extended prompt
To create the prompt we use the result from query the Vector Database  and the sentence introduced by the user. 

The prompt have two parts, the **relevant context** that is the information recovered from the database and the **user's question**. 

We only need to join the two parts together to create the prompt that we are going to send to the model. 

You can limit the lenght of the context passed to the model, because we can get some Memory problems with one of the datasets that contains a realy large text in the document part. 

In [20]:
question = "Can I buy a Toshiba laptop?"
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
#context = context[0:5120]
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"
prompt_template

"Relevant context: #Photography student Thorsten Mjölnir captures the way students decorate their laptops. #Why sales of very basic mobile phones, without apps and internet connection, are increasing. #What do you do when your collection of millions of books keeps growing but your bookshelves don't? #The developers of a powerful mini aircraft hope it will be used by the armed forces. #How tech is helping young families and couples regain their busy social lives after Covid. #Watch as Lee Zii Jia of Malaysia records a speed of 372km/h on his backhand point against Lakshya Sen of India in the All England Badminton Championships. #The Royal Mint has found a way to turn old circuit boards from phones, computers and TVs into gold. #A van was reportedly hijacked and driven to the venue, and a controlled explosion has since been carried out. #The Ukrainian president reveals his location in Kyiv in a new video shared on social media. #The Royal Mint has found a way to turn old circuit boards f

Now all that remains is to send the prompt to the model and wait for its response!


In [21]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

Relevant context: #Photography student Thorsten Mjölnir captures the way students decorate their laptops. #Why sales of very basic mobile phones, without apps and internet connection, are increasing. #What do you do when your collection of millions of books keeps growing but your bookshelves don't? #The developers of a powerful mini aircraft hope it will be used by the armed forces. #How tech is helping young families and couples regain their busy social lives after Covid. #Watch as Lee Zii Jia of Malaysia records a speed of 372km/h on his backhand point against Lakshya Sen of India in the All England Badminton Championships. #The Royal Mint has found a way to turn old circuit boards from phones, computers and TVs into gold. #A van was reportedly hijacked and driven to the venue, and a controlled explosion has since been carried out. #The Ukrainian president reveals his location in Kyiv in a new video shared on social media. #The Royal Mint has found a way to turn old circuit boards fr